In [ ]:
from tqdm import tqdm

HAE-RAE 데이터들이 Parquet 포맷으로 제공되어서 먼저 csv로 변환해준다


In [ ]:
import os

import pandas as pd

# 변환할 폴더 경로
input_folder = "./data/agg_other_benchmarks/HAE-RAE"  # Parquet 파일이 있는 폴더
output_folder = "./data/agg_other_benchmarks/HAE-RAE"  # Parquet 파일 저장 폴더

# 폴더 내 모든 파일 처리
for file_name in os.listdir(input_folder):
    if file_name.endswith(".parquet"):  # Parquet 파일만 선택
        input_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, file_name.replace(".parquet", ".csv"))

        # Parquet → CSV 변환
        try:
            df = pd.read_parquet(input_path, engine="pyarrow")  # Parquet 파일 읽기
            df.to_csv(output_path, index=False, encoding="utf-8")  # CSV 파일 저장
            print(f"변환 완료: {file_name} → {output_path}")
        except Exception as e:
            print(f"변환 실패: {file_name} - {e}")

변환 완료: standard_nomenclature-00000-of-00001-52814989a6843328.parquet → ./data/agg_other_benchmarks/HAE-RAE/standard_nomenclature-00000-of-00001-52814989a6843328.csv
변환 완료: csat_law-00000-of-00001-ef0ec0fac2f2238f.parquet → ./data/agg_other_benchmarks/HAE-RAE/csat_law-00000-of-00001-ef0ec0fac2f2238f.csv
변환 완료: lyrics_denoising-00000-of-00001-ede1ce975e9bad54.parquet → ./data/agg_other_benchmarks/HAE-RAE/lyrics_denoising-00000-of-00001-ede1ce975e9bad54.csv
변환 완료: general_knowledge-00000-of-00001-7418d1a0b8c4a615.parquet → ./data/agg_other_benchmarks/HAE-RAE/general_knowledge-00000-of-00001-7418d1a0b8c4a615.csv
변환 완료: history-00000-of-00001-9ad551d475d58413.parquet → ./data/agg_other_benchmarks/HAE-RAE/history-00000-of-00001-9ad551d475d58413.csv
변환 완료: rare_words-00000-of-00001-4f34314c09209524.parquet → ./data/agg_other_benchmarks/HAE-RAE/rare_words-00000-of-00001-4f34314c09209524.csv
변환 완료: csat_socio-00000-of-00001-0cfadd93827d0d48.parquet → ./data/agg_other_benchmarks/HAE-RAE/csat_soc

기존 Train 데이터의 형식은 다음과 같다
id,paragraph,question,choices,answer,question_plus

HAE-RAE 데이터는
query,options,answer 와 같은 형식으로 구성 돼 있다.
query 안에 지문, 질문, 선택지가 있고, options 안에 선택지가 또 있다.
answer은 알파벳 A,B,C,D,E로 돼있다.


취합을 위한 데이터 가공 작업의 순서

- query 안에서 지문, 질문, 선택지를 분리한다.
- 질문은 question 칼럼에 옮겨준다
- 선택지 부분은 지워준다 (options 부분에 있으므로)
- query 안에는 지문만 남겨준다

- answer은 각 알파벳을 숫자로 변환해준다 (A는 1, B는 2, C는 3...)


In [ ]:
# HAERAE 의 독해 데이터셋을 불러와준다

import pandas as pd

data_haerae_rc = pd.read_csv("./data/agg_other_benchmarks/pending/HAE-RAE_reading_comprehension.csv")

query안의 문자열은 다음과 같이 구분 돼 있다.

### 지문:

### 질문:

### 선택지:

이를 이용하여 지문만 남기고, 질문은 question 칼럼으로 옮겨주고 선택지 부분은 지우겠다.


In [ ]:
# 정규표현식으로 특정 패턴의 텍스트를 추출하는 함수를 선언해준다

import re


def extract_after(text, keyword):
    pattern = f"{re.escape(keyword)}(.*)"  # 키워드 이후 모든 문자열 매칭
    match = re.search(pattern, text)
    if match:
        return match.group(1).strip()  # 키워드 이후 문자열 반환
    return None  # 키워드가 없으면 None 반환


def extract_between(text, start_keyword, end_keyword):
    # 시작과 끝 키워드 사이의 텍스트 추출
    pattern = f"{re.escape(start_keyword)}(.*?){re.escape(end_keyword)}"
    match = re.search(pattern, text, re.DOTALL)  # re.DOTALL은 줄바꿈 포함 매칭
    if match:
        return match.group(1).strip()
    return None


# 알파벳을 숫자로 맵핑하는 함수를 만들어준다 (answer 칼럼용)
def convert_string_to_number(s):
    # 문자열-숫자 매핑 딕셔너리
    mapping = {"(A)": 1, "(B)": 2, "(C)": 3, "(D)": 4, "(E)": 5}
    # 매핑에 없는 문자열은 None 반환
    return mapping.get(s, None)

In [ ]:
train_agg = pd.read_csv("./data/agg_other_benchmarks/train_agg.csv")
data_haerae_rc_agg = pd.DataFrame(columns=train_agg.columns)

In [ ]:
count = 0

for i in tqdm(range(len(data_haerae_rc))):
    id = f"haerae-rc-{count}"
    query = data_haerae_rc.iloc[i]["query"]
    passage = extract_between(query, "### 지문:", "### 질문:")
    question = extract_between(query, "### 질문:", "### 선택지:")
    choices = str(data_haerae_rc.iloc[i]["options"])
    answer = convert_string_to_number(data_haerae_rc.iloc[i]["answer"])
    df_agg = pd.DataFrame(
        [
            {
                "id": id,
                "paragraph": passage,
                "question": question,
                "choices": choices,
                "answer": answer,
                "question_plus": None,
            }
        ]
    )
    train_agg = pd.concat([train_agg, df_agg], ignore_index=True)
    count += 1

  0%|          | 0/936 [00:00<?, ?it/s]

100%|██████████| 936/936 [00:01<00:00, 884.19it/s]


In [ ]:
train_agg.to_csv("train_agg_haerae_rc.csv", index=False)